### Add to map research
    - Add Data widget (portal, url)
    - How's My Waterway (portal, url, file)
    - Summarize My Area

#### Add Data widget

#### Map Viewer Browse for layers...addfromonline.js
    - async function with try/catch logic 
    - incorporates whenOnce() method 
        - https://developers.arcgis.com/javascript/latest/api-reference/esri-core-reactiveUtils.html#whenOnce

In [ ]:
                    async addItemToMap() {
                        try {
                            var d = this.itemBrowserSelectedItem || this.itemBrowserPreviewSelectedItem;
                            let k = await q.toLayer(d, this.isFromAGOL(d), this.state);
                            await w.default.whenOnce( () => k.layerId || k.loaded || k.loadError);
                            const x = !this.itemBrowserSelectedItem && this.itemBrowserSelectedSublayerItem;
                            x && ({id: d} = x,
                            k = k.layerId === d ? k : await this.getSublayer(d, k));
                            await q.addLayers([k], this.state);
                            w.default.whenOnce( () => k.isTable || k.fullExtent || k.loadError || k.loaded).then( () => {
                                k.isTable || k.loadError || this.gotoFirstLayer([k], this.state)
                            }
                            );
                            return k
                        } catch (k) {
                            return Promise.reject(k)
                        }
                    }

#### Featured Collections (webmap)
    - get layers from a web map

In [ ]:
_onConfirmation : function(item) {
    # this.promptUserToZoomToItem(item);
    var self = this;
    this.zoomToItem(item);
    showLayerListWidget();
    testmap = this.map;
    tempLayer = false;
    # use item.tags and window.communityDic to get the community info
            var bIsTaggedCommunity = false;
            for (var i in item.tags) {
                if (window.communityDic.hasOwnProperty(item.tags[i])) {
                    bIsTaggedCommunity = true;
                    window.communitySelected = item.tags[i];
                    break;
                }
            }
            if (bIsTaggedCommunity) {
            	setTimeout(function () {  
	                selfAddWebMapData.publishData({
	                    message : "updateCommunityLayers"
	                });
                }, 3000)
            }//
            
            /*//copied from original widget
            var mapConfig = {
                    "itemId": item.id
                };
                // this is the official way to do this, but reloads the whole app instead of just the map
                //MapManager.getInstance().onAppConfigChanged(this.appConfig, 'mapChange', mapConfig);
                ConfigManager.getInstance()._onMapChanged(mapConfig);*/

            //get all LayerId before adding layers

            item.getItemData().then(function(response) {
                //process operational layers in reverse order to match AGOL
                layersReversed = response.operationalLayers.reverse();
                                
                //first push Feature layers so that it will be added at last
                layersReversed.forEach(function(l) {
                    if (l.url) {
                        if (l.id && (l.id != undefined)) {
                            if (l.layerType == 'ArcGISFeatureLayer'){
                                if (arrlayerId.indexOf(l.id) < 0) {
                                    arrlayerId.push(l.id);
                                }
                            }
                        }
                    }
                });
                
                
                //layersReversed = response.operationalLayers;
                //then push Map service layers so that it will be added before Feature layers
                layersReversed.forEach(function(l) {
                    if (l.url) {
                        if (l.id && (l.id != undefined)) {
                            if (l.layerType == 'ArcGISMapServiceLayer' || l.layerType == 'ArcGISImageServiceLayer' || l.layerType == 'ArcGISTiledMapServiceLayer'){
                                if (arrlayerId.indexOf(l.id) < 0) {
                                    arrlayerId.push(l.id);
                                }
                            }
                        }
                    }
                });

                
                addAddItemToMapOperational(layersReversed, arrlayerId.pop(), response);

            })


            setTimeout(lang.hitch(this, function() {
                item.getItemData().then(function(response) {
                    //process operational layers in reverse order to match AGOL
                    layersReversed = response.operationalLayers.reverse();
                    layersReversed.forEach(function(l) {
                        if (l.url) {
                        } else {
                            if (l.featureCollection) {
                                console.log("Web Map Layers:: FeatureCollection");
                                l.featureCollection.layers.forEach(function(subL) {
                                    tempLayer = new FeatureLayer(subL, {
                                        id : l.id
                                    });
                                    tempLayer = selfAddWebMapData._processLayer(tempLayer, subL);
                                });
                            } else {
                                console.log("Add Layer Error:: Layer of unknown type");
                                if (!(l.url in window.faildedEALayerDictionary)) {
                                    window.faildedEALayerDictionary[l.url] = l.url;
                                    selfAddWebMapData.publishData({
                                        message : "openFailedLayer"
                                    });
                                }
                            }
                        }
                        if (tempLayer) {
                            //tempLayer.title = l.title;
                            window.layerID_Portal_WebMap.push(l.id);
                            testmap.addLayer(tempLayer);
                        }
                    });
                });
            }), 4000);
            var widgetManager;
            var fcDetailsWidgetEle = selfAddWebMapData.appConfig.getConfigElementsByName("FeaturedCollectionPreview")[0];
            widgetManager = WidgetManager.getInstance();
            widgetManager.closeWidget(fcDetailsWidgetEle.id);
            document.getElementById("titleForFCWidget").style.display = "none";
            document.getElementById("closeFCWidgetArea").style.display = "none";
            window.fcDetailsOpened = false;
        },

In [ ]:
_processLayer : function(tempLayer, l) {
            // Borrowed from AddData/search/LayerLoader.js _processFeatureLayer
            var layerDefinition, renderer = false;
            
        	var tileURL = "";
        	var lebURL = "";
        	var eaIDinSearchFilter = "";
        	if (window.hashURLtoTile.hasOwnProperty(l.url)) {
        		tileURL = window.hashURLtoTile[l.url];
        	}
        	else { 
        		lebURL = l.url.replace("enviroatlas.epa.gov", "leb.epa.gov");
        		if(window.hashURLtoTile.hasOwnProperty(lebURL)) {
        			tileURL = window.hashURLtoTile[lebURL];                	
        		}                		
        	}


   	       for (var key in window.hashURL){//window.hashURL[layer.eaID.toString()] = eaURL; 
			  if ((window.hashURL[key]==l.url) || (window.hashURL[key]==lebURL)) {
			  	eaIDinSearchFilter = key;
			  	window.hashFeaturedCollectionToEAID[l.id] = eaIDinSearchFilter;  
			  }
			}
 
        	if (tileURL=="") {
	            var popInfo, infoTemplate;
	            if (l.popupInfo) {
	                popInfo = l.popupInfo;
	                jsonPopInfo = djJson.parse(djJson.stringify(popInfo));
	                infoTemplate = new PopupTemplate(jsonPopInfo);
	                tempLayer.setInfoTemplate(infoTemplate);
	            }
           } else {//Its url matches the EnviroAtlas layer          	
	
   	            window.featureLyrNumber.push(eaIDinSearchFilter); 
   	            // add corresponding tile            
            	lOptions = {};
            	lOptions.visible = l.visibility;
            	lOptions.opacity = l.opacity;
                lOptions.id = window.layerIdTiledPrefix + eaIDinSearchFilter;
                

                this.map.addLayer(new ArcGISTiledMapServiceLayer(tileURL, lOptions));  
                             
           }
           
            if (esriLang.isDefined(l.showLabels)) {
                tempLayer.setShowLabels(l.showLabels);
            }
            if (esriLang.isDefined(l.refreshInterval)) {
                tempLayer.setRefreshInterval(l.refreshInterval);
            }
            if (esriLang.isDefined(l.showLegend)) {
                // TODO?
                console.log('');
            }
            if (esriLang.isDefined(l.timeAnimation)) {
                if (l.timeAnimation === false) {
                    // TODO?
                    console.log("");
                }
            }
            layerDefinition = l.layerDefinition;
            if (layerDefinition) {
                console.log("layerDefinition",layerDefinition)
                if (layerDefinition.definitionExpression) {
                    tempLayer.setDefinitionExpression(layerDefinition.definitionExpression);
                }
                if (layerDefinition.displayField) {
                    tempLayer.displayField(layerDefinition.displayField);
                }
                if (layerDefinition.drawingInfo) {
                    if (layerDefinition.drawingInfo.renderer) {
                        jsonRenderer = djJson.parse(djJson.stringify(layerDefinition.drawingInfo.renderer));
                        renderer = jsonRendererUtils.fromJson(jsonRenderer);
                        if (jsonRenderer.type && (jsonRenderer.type === "classBreaks")) {
                            renderer.isMaxInclusive = true;
                        }
                        tempLayer.setRenderer(renderer);
                    }
                    if (esriLang.isDefined(layerDefinition.drawingInfo.transparency)) {
                        // TODO validate before setting?
                        tempLayer.setOpacity(1 - (layerDefinition.drawingInfo.transparency / 100));
                    }
                }

                if (esriLang.isDefined(layerDefinition.minScale)) {
                    tempLayer.setMinScale(layerDefinition.minScale);
                }
                if (esriLang.isDefined(layerDefinition.maxScale)) {
                    tempLayer.setMaxScale(layerDefinition.maxScale);
                }
                if (esriLang.isDefined(layerDefinition.defaultVisibility)) {
                    if (layerDefinition.defaultVisibility === false) {
                        tempLayer.setVisibility(false);
                        // TODO?
                    }
                }
            }
            return tempLayer;
        }

In [ ]:
var addAddItemToMapOperational = function(layersReversed, layerId, response) {
        layersReversed.forEach(function(l) {
            if ((l.url) && (l.id == layerId)) {

                if (l.layerType == 'ArcGISTiledMapServiceLayer') {
                    tempLayer = new ArcGISTiledMapServiceLayer(l.url, {
                        id : l.id,
                        opacity : l.opacity,
                        visible : l.visibility
                    });
                    tempLayer.title = l.title;
                }else if (l.layerType == 'ArcGISImageServiceLayer') {
                    tempLayer = new ArcGISImageServiceLayer(l.url, {
                        id : l.id,
                        opacity : l.opacity,
                        visible : l.visibility
                    });
                    tempLayer.title = l.title;
                    tempLayer = selfAddWebMapData._processLayer(tempLayer, l);
                } else if (l.layerType == 'ArcGISFeatureLayer') {
                    tempLayer = new FeatureLayer(l.url, {
                        mode : FeatureLayer.MODE_ONDEMAND,
                        id : l.id,
                        opacity : l.opacity,
                        visible : l.visibility,
                        outFields : ["*"],
                    });
                    tempLayer.title = l.title;
                    tempLayer = selfAddWebMapData._processLayer(tempLayer, l);
                } else if (l.layerType == 'ArcGISMapServiceLayer') {
                    tempLayer = new ArcGISDynamicMapServiceLayer(l.url, {
                        id : l.id,
                        opacity : l.opacity,
                        visible : l.visibility,
                        "showAttribution" : false
                    });
           
                    tempLayer.title = l.title;
                    //if layers have popupInfo grab them
                    if (l.layers) {
                        var expressions = [];
                        var dynamicLayerInfo;
                        var dynamicLayerInfos = [];
                        var drawingOptions;
                        var drawingOptionsArray = [];
                        var source;
                        var _infoTemps = {};
                        var indexTemplate = 0;
                        array.forEach(l.layers, function(layerInfo) {
                            if (layerInfo.layerDefinition && layerInfo.layerDefinition.definitionExpression) {
                                expressions[layerInfo.id] = layerInfo.layerDefinition.definitionExpression;
                            }
                            //get infoTemplate
                            /*if (layerInfo.popupInfo) {
                                popInfo = layerInfo.popupInfo;
                                jsonPopInfo = djJson.parse(djJson.stringify(popInfo));
                                //infoTemplate = new PopupTemplate(jsonPopInfo);
                                //var json = {title:popInfo.title,
                                //     content:  popInfo.description.substring(1,popInfo.description.length - 1) + ":" + "$"+ popInfo.description//"State Name: ${STATE_NAME}"
                                //}
                                //infoTemplate = new PopupTemplate(json);

                                infoTemplate = new PopupTemplate({
                                    title : layerInfo.name,
                                    fieldInfos : [{
                                        fieldName : popInfo.description.substring(1, popInfo.description.length - 1),
                                        visible : true,
                                        label : "",
                                        content : "$" + popInfo.description
                                    }]
                                })
                                _infoTemps[indexTemplate] = {};
                                _infoTemps[indexTemplate]["infoTemplate"] = infoTemplate;

                                //tempLayer.setInfoTemplate(infoTemplate);
                            }*/
                            //end of getting infoTemplate
                            if (layerInfo.layerDefinition && layerInfo.layerDefinition.source) {
                                dynamicLayerInfo = null;
                                source = layerInfo.layerDefinition.source;
                                if (source.type === "mapLayer") {

                                    var metaLayerInfos = array.filter(response.operationalLayers, function(rlyr) {
                                        return rlyr.id === source.mapLayerId;
                                    });
                                    if (metaLayerInfos.length) {
                                        dynamicLayerInfo = lang.mixin(metaLayerInfos[0], layerInfo);
                                    }
                                } else {
                                    dynamicLayerInfo = lang.mixin({}, layerInfo);
                                }
                                if (dynamicLayerInfo) {
                                    dynamicLayerInfo.source = source;
                                    delete dynamicLayerInfo.popupInfo;
                                    dynamicLayerInfo = new DynamicLayerInfo(dynamicLayerInfo);
                                    if (l.visibleLayers) {
                                        var vis = (( typeof l.visibleLayers) === "string") ? l.visibleLayers.split(",") : l.visibleLayers;
                                        if (array.indexOf(vis, layerInfo.id) > -1) {
                                            dynamicLayerInfo.defaultVisibility = true;
                                        } else {
                                            dynamicLayerInfo.defaultVisibility = false;
                                        }
                                    }
                                    dynamicLayerInfos.push(dynamicLayerInfo);
                                }
                            } 
                            if (layerInfo.layerDefinition && layerInfo.layerDefinition.source && layerInfo.layerDefinition.drawingInfo) {
                                drawingOptions = new LayerDrawingOptions(layerInfo.layerDefinition.drawingInfo);
                                drawingOptionsArray[layerInfo.id] = drawingOptions;
                            }
                            indexTemplate = indexTemplate + 1;
                        });
                        tempLayer.setInfoTemplates(_infoTemps);

                        if (expressions.length > 0) {
                            tempLayer.setLayerDefinitions(expressions);
                        }
                        if (dynamicLayerInfos.length > 0) {
                            tempLayer.setDynamicLayerInfos(dynamicLayerInfos, true);
                        }
                        if (drawingOptionsArray.length > 0) {
                            tempLayer.setLayerDrawingOptions(drawingOptionsArray, true);
                        }

                    } else if (l.visibleLayers) {
                        tempLayer.setVisibleLayers(l.visibleLayers);
                    }

                }
            }

            if (tempLayer) {
                tempLayer.on('load', function(evt) {
                    var chkToBeClicked =  null;
                        
                    var i = 0;                     //  set your counter to 1
                    var maximumCount = 10;                    
                                 
                    if (arrlayerId.length > 0) {
                        setTimeout(lang.hitch(this, function() {
                            addAddItemToMapOperational(layersReversed, arrlayerId.pop(), response);
                        }), 10);
                    }
                });
                window.layerID_Portal_WebMap.push(l.id);
                testmap.addLayer(tempLayer);
            }
        });

    };

#### Time Series (image server)
    - new imageservice layer, set properties, add to map
    - after on map, open layer list widget
    - specify metadata?
    - add data using url parameter

In [ ]:
        var imageServiceLayer = new ArcGISImageServiceLayer(selectedImageService,{imageServiceParameters: params});
        imageServiceLayer.id = window.timeSeriesLayerId;
        imageServiceLayer.name = "TimeSeries_" + modelValue + "_" + season + "_" + climateVar;
        imageServiceLayer.title = "TimeSeries_" + modelValue + "_" + season + "_" + climateVar;
        imageServiceLayer.setOpacity(0.6);
        // window.climateTimeSeriesFromURL = imageServiceLayer.url
        
        map.addLayers([imageServiceLayer]);
        
        //Turn on Identify capability after the layer is added to the map
        mapClickListener = map.on("click", executeIdentifyTask);
        
        imageServiceLayer.on("load", function(){
            //var modelId = dijit.byId("modelSelection").item.id;
	        var unit = "";
            if ((climateVar == "TempMax") ||(climateVar == "TempMin")) {
            	unit = "Degrees F";                
            } else if ((climateVar == "Precip") ||(climateVar == "PET")) {
            	unit = "Inches";                
            } 	   
	        //setMetadataTab(modelId + " (" + modelValue + "), " + season + " <br/>" + climateId + " (" + unit + ")<br/><hr>" + comment);
	        setMetadataTab(modelValue + ", " + season + " <br/>" + climateVar + " (" + unit + ")<br/><hr>" + comment);           
	        showLayerListWidget();       

#### Simple Search Filter
    - layer.types = [IMAGE, WEBTILEDLAYER, WEBTILEDBASEMAP, FEATURE, TILED, WMS]
    - scale levels for tile vs feature
    - set popup template ("showAttachements", "description" no longer a prop)

In [ ]:
var _addSelectedLayers = function (layersTobeAdded, selectedLayerNum) {
			var index,
				len;
			var selectedLayerArray = selectedLayerNum.split(",");
			for (i in selectedLayerArray) {
				for (index = 0,
					len = layersTobeAdded.length; index < len; ++index) {
					layer = layersTobeAdded[index];
					if (layer.hasOwnProperty('eaID') && ((selectedLayerArray[i]) == (layer.eaID.toString()))) {
						var bNeedToBeAdded = true;
						var lLayer;
						var lOptions = {};
						if (layer.hasOwnProperty('opacity')) {
							lOptions.opacity = layer.opacity;
							// 1.0 has no transparency; 0.0 is 100% transparent
						}
						/*if (layer.hasOwnProperty('visible') && !layer.visible) {
							lOptions.visible = false;
						} else {
							lOptions.visible = true;
						}*/
						lOptions.visible = true;
						if (layer.name) {
							lOptions.id = layer.name;
						}
						if (layer.hasOwnProperty('hidelayers')) {
							if (layer.hidelayers) {
								lOptions.hidelayers = [];
								lOptions.hidelayers = layer.hidelayers.split(',');
							}
						}
						if (layer.hasOwnProperty('drawSelectLayer')) {
							if (layer.drawSelectLayer) {
								lOptions.visiblelayers = layer.drawSelectLayer.map(Number);
							}
						}
						if (layer.hasOwnProperty('minScale')) {
							lOptions.minScale = layer.minScale
						}
						if (layer.hasOwnProperty('maxScale')) {
							lOptions.maxScale = layer.maxScale
						}
						if (layer.type.toUpperCase() === 'DYNAMIC') {
							window.dynamicLayerNumber.push(layer.eaID);
							if (layer.imageformat) {
								var ip = new ImageParameters();
								ip.format = layer.imageformat;
								if (layer.hasOwnProperty('imagedpi')) {
									ip.dpi = layer.imagedpi;
								}
								lOptions.imageParameters = ip;
							}
							lLayer = new ArcGISDynamicMapServiceLayer(layer.url, lOptions);

							if (layer.hasOwnProperty('definitionQueries')) {
								var definitionQueries = JSON.parse(layer.definitionQueries)
								var layerDefinitions = []
								for (var prop in definitionQueries) {
									layerDefinitions[prop] = definitionQueries[prop];
								}
								lLayer.setLayerDefinitions(layerDefinitions);
							}
							if (layer.name) {
								lLayer._titleForLegend = layer.name;
								lLayer.title = layer.name;
								window.hashTitleToEAID[layer.name] = layer.eaID;
								window.hashEAIDToTitle[layer.eaID] = layer.name;
								lLayer.noservicename = true;
							}
							var popupConfig = jimuUtils.getPopups(layer);
							lLayer.setInfoTemplates(popupConfig);

							if (layer.hasOwnProperty('autorefresh')) {
								lLayer.refreshInterval = layer.autorefresh;

							}
							if (layer.disableclientcaching) {
								lLayer.setDisableClientCaching(true);
							}
							lLayer.on('error', function (evt) {
								console.log(evt);
							})

							lLayer.on('load', function (evt) {
								if (layer.flyPopups) {
									var _infoTemps = []
									evt.layer.layerInfos.forEach(function (layer) {
										_infoTemps.push({
											infoTemplate: new PopupTemplate({
												title: layer.name,
												fieldInfos: [{
													fieldName: "*",
													visible: true,
													label: "*"
												}]
											})
										})
									})
									evt.layer.setInfoTemplates(_infoTemps)
								}
								//set min/max scales if present
								if (lOptions.minScale) {
									evt.layer.setMinScale(lOptions.minScale)
								}
								if (lOptions.maxScale) {
									evt.layer.setMaxScale(lOptions.maxScale)
								}

								if (!lOptions.hasOwnProperty('hidelayers')) {
									lOptions.hidelayers = []
								}
								var removeLayers = []
								for (var i = 0; i < lOptions.hidelayers.length; i++) {
									lOptions.hidelayers[i] = parseInt(lOptions.hidelayers[i])
								}
								var showLayers = []
								array.forEach(evt.layer.layerInfos, function (layer) {
									showLayers.push(layer.id)
								})
								array.forEach(lOptions.hidelayers, function (id) {
									showLayers.splice(showLayers.indexOf(id), 1)
								})
								lOptions.hidelayers = showLayers
								var getArrayItemById = function (_array, _id) {
									var _matchItem;
									array.some(_array, function (_arrayItem) {
										if (_arrayItem.id == _id) {
											_matchItem = _arrayItem;
											return true;
										}
									})
									return _matchItem;
								}
								array.forEach(evt.layer.layerInfos, function (layer) {
									layer.defaultVisibility = false;
								})
								for (var i = 0; i < lOptions.hidelayers.length; i++) {
									getArrayItemById(evt.layer.layerInfos, lOptions.hidelayers[i]).defaultVisibility = true;
								}
								array.forEach(evt.layer.layerInfos, function (layer) {
									if (layer.subLayerIds) {
										if (removeLayers.indexOf(layer.id) == -1) {
											removeLayers.push(layer.id)
										};
									}
								})
								for (var i = 0; i < lOptions.hidelayers.length; i++) {
									var j = getArrayItemById(evt.layer.layerInfos, lOptions.hidelayers[i]).parentLayerId
									while (j > -1) {
										if (lOptions.hidelayers.indexOf(j) == -1) {
											if (removeLayers.indexOf(lOptions.hidelayers[i]) == -1) {
												removeLayers.push(lOptions.hidelayers[i])
											}
										}
										j = getArrayItemById(evt.layer.layerInfos, j).parentLayerId;
									}
								}
								array.forEach(removeLayers, function (layerId) {
									if (lOptions.hidelayers.indexOf(layerId) > -1) {
										lOptions.hidelayers.splice(lOptions.hidelayers.indexOf(layerId), 1)
									};
								})
								if (lOptions.hidelayers.length == 0) {
									lOptions.hidelayers.push(-1);
									lOptions.hidelayers.push(-1);
									lOptions.hidelayers.push(-1);
								}

								//evt.layer.setVisibleLayers(lOptions.hidelayers);
								if (lOptions.visiblelayers) {
									evt.layer.setVisibleLayers([-1], true);
									setTimeout(lang.hitch(this, function () {
										evt.layer.setVisibleLayers(lOptions.visiblelayers, true);
										//evt.layer.setVisibleLayers([12],true);
									}), 2000);
								}
								if (layer.hasOwnProperty('hideInLegends')) {
									var hideLegends = JSON.parse(layer.hideInLegends)
									var finalLegends = []
									for (var prop in hideLegends) {
										array.forEach(evt.layer.layerInfos, lang.hitch(this, function (layerInfo) {
											if (layerInfo.id == parseInt(prop)) {
												layerInfo.showLegend = !hideLegends[prop]
											}
										}))
									}
								}
								lLayer.layers = evt.layer.layerInfos
							});

							this._viewerMap.setInfoWindowOnClick(true);
						} else if (layer.type.toUpperCase() === 'IMAGE') {
							window.imageLayerNumber.push(layer.eaID);
							lOptions.imageServiceParameters = new ImageServiceParameters();
							var _popupTemplate;
							if (layer.popup) {
								_popupTemplate = new PopupTemplate(layer.popup);
								lOptions.infoTemplate = _popupTemplate;
							}
							lLayer = new ArcGISImageServiceLayer(layer.url, lOptions)
							if (layer.hasOwnProperty('hideInLegend')) {
								lLayer.showLegend = !layer.hideInLegend
							}
							if (layer.name) {
								lLayer._titleForLegend = layer.name;
								lLayer.title = layer.name;
								window.hashTitleToEAID[layer.name] = layer.eaID;
								window.hashEAIDToTitle[layer.eaID] = layer.name;
								lLayer.noservicename = true;
							}
							lLayer.on('load', function (evt) {
								if (lOptions.minScale) {
									evt.layer.setMinScale(lOptions.minScale)
								}
								if (lOptions.maxScale) {
									evt.layer.setMaxScale(lOptions.maxScale)
								}
								evt.layer.name = lOptions.id;
							});
							//_layersToAdd.push(lLayer);
						} else if (layer.type.toUpperCase() === 'WEBTILEDLAYER') {
							if (layer.hasOwnProperty('subdomains')) {
								lOptions.subDomains = layer.subdomains;
							}
							if (layer.hasOwnProperty('autorefresh')) {
								lOptions.refreshInterval = layer.autorefresh;
							}
							if (layer.hasOwnProperty('opacity')) {
								lOptions.opacity = layer.opacity;
							}
							lLayer = new WebTiledLayer(layer.url, lOptions)
							lLayer.on('load', function (evt) {
								if (lOptions.minScale) {
									evt.layer.setMinScale(lOptions.minScale)
								}
								if (lOptions.maxScale) {
									evt.layer.setMaxScale(lOptions.maxScale)
								}
								evt.layer.name = lOptions.id;
							});
							_layersToAdd.push(lLayer);
						} else if (layer.type.toUpperCase() === 'WEBTILEDBASEMAP') {
							lOptions.type = "WebTiledLayer"
							lOptions.url = layer.url
							if (layer.hasOwnProperty('subdomains')) {
								lOptions.subDomains = layer.subdomains;
							}
							if (layer.hasOwnProperty('autorefresh')) {
								lOptions.refreshInterval = layer.autorefresh;
							}
							if (layer.hasOwnProperty('opacity')) {
								lOptions.opacity = layer.opacity;
							}
							if (layer.hasOwnProperty('copyright')) {
								lOptions.copyright = layer.copyright;
							}
							var _newBasemap = new Basemap({
								id: 'defaultBasemap',
								title: layer.name,
								layers: [new BasemapLayer(lOptions)]
							});
							var _basemapGallery = new BasemapGallery({
								showArcGISBasemaps: false,
								map: this._viewerMap
							}, '_tmpBasemapGallery');
							_basemapGallery.add(_newBasemap);
							_basemapGallery.select('defaultBasemap');
							_basemapGallery.destroy();
						} else if (layer.type.toUpperCase() === 'FEATURE') {
							window.featureLyrNumber.push(layer.eaID);
							bPopup = true;
							var _popupTemplate;
							if (layer.popup) {
								window.hashPopup[layer.eaID] = layer.popup;
								if (layer.popup.fieldInfos) {
									fieldInfos = layer.popup.fieldInfos;
									if (fieldInfos[0].hasOwnProperty('fieldName')) {
										if (fieldInfos[0].fieldName == null) {
											bPopup = false;
										} else {
											Attribute = fieldInfos[0].fieldName;
											hashAttribute[layer.eaID.toString()] = Attribute;//It will be used as window.hashAttribute
										}
									} else {
										bPopup = false;
									}
								} else {
									bPopup = false;
								}
								if (bPopup) {
									_popupTemplate = new PopupTemplate(layer.popup);
									//lOptions.infoTemplate = _popupTemplate;
								} else {
									console.log("layer.eaID: " + +layer.eaID.toString() + " with no popup info defined");
								}
							}
							if (layer.hasOwnProperty('mode')) {
								var lmode;
								if (layer.mode === 'ondemand') {
									lmode = 1;
								} else if (layer.mode === 'snapshot') {
									lmode = 0;
								} else if (layer.mode === 'selection') {
									lmode = 2;
								}
								lOptions.mode = lmode;
							}
							lOptions.outFields = ['*'];
							if (layer.hasOwnProperty('autorefresh')) {
								lOptions.refreshInterval = layer.autorefresh;
							}
							if (layer.hasOwnProperty('showLabels')) {
								lOptions.showLabels = true;
							}

							if (bPopup) {
								if (layer.hasOwnProperty('eaLyrNum')) {
									lLayer = new FeatureLayer(layer.url + "/" + layer.eaLyrNum.toString(), lOptions);
									window.hashURL[layer.eaID] = layer.url + "/" + layer.eaLyrNum.toString();
								} else {
									lLayer = new FeatureLayer(layer.url, lOptions);
								}
							}

							if (bNeedToBeAdded) {
								if (layer.tileLink == "yes") {
									var tileLinkAdjusted = "";
									if (layer.tileURL.slice(-1) == "/") {
										tileLinkAdjusted = layer.tileURL;
									} else {
										tileLinkAdjusted = layer.tileURL + "/";
									}
									window.hashIDtoTileURL[layer.eaID.toString()] = tileLinkAdjusted;
									if (tileLinkAdjusted.slice(-11) == "_alllayers/") {
										jimuUtils.initTileLayer(tileLinkAdjusted, window.layerIdTiledPrefix + layer.eaID.toString());
										this._viewerMap.addLayer(new myTiledMapServiceLayer());
										lyrTiled = this._viewerMap.getLayer(window.layerIdTiledPrefix + layer.eaID.toString());
										//bji need to be modified to accomodate tile.
										if (lyrTiled) {
											lyrTiled.setOpacity(layer.opacity);
										}
									}
									else {
										lOptions.id = window.layerIdTiledPrefix + layer.eaID.toString();
										var tileLayerForFeature = new ArcGISTiledMapServiceLayer(layer.tileURL, lOptions);
										//tileLayerForFeature.setMaxScale(2000000);
										if (layer.hasOwnProperty('tilecacheLevelNat')) {
											tileLayerForFeature.setMaxScale(Math.ceil(window.scaleLevelDic[layer.cacheLevelNat]));
										}
										/*tileLayerForFeature.on('load', function(evt) {
												setTimeout(function () {
													evt.layer.setMaxScale(2000000);     
												}, 100)                      			                            
	
										});*/
										this._viewerMap.addLayer(tileLayerForFeature);

									}
								} else if (layer.eaScale == "COMMUNITY") {
									loadSymbologyConfig(function (response) {
										var classBreakInfo = JSON.parse(response);
										var renderer = new ClassBreaksRenderer(classBreakInfo);
										lLayer.setRenderer(renderer);
									});
								}
							}
						} else if (layer.type.toUpperCase() === 'TILED') {
							window.tiledLayerNumber.push(layer.eaID);
							if (layer.displayLevels) {
								lOptions.displayLevels = layer.displayLevels;
							}
							if (layer.hasOwnProperty('autorefresh')) {
								lOptions.refreshInterval = layer.autorefresh;
							}
							lLayer = new ArcGISTiledMapServiceLayer(layer.url, lOptions);

							var popupConfig = jimuUtils.getPopups(layer);
							lLayer.setInfoTemplates(popupConfig);

						} else if (layer.type.toUpperCase() === "WMS") {

							lLayer = new WMSLayer(layer.url, {
								format: "png",
							});

						}
						//All layer types:
						if (bNeedToBeAdded) {
							dojo.connect(lLayer, "onError", function (error) {
								if ((!(lLayer.title in window.faildedEALayerDictionary)) && (!(lLayer.title in window.successLayerDictionary))) {
									window.faildedEALayerDictionary[lLayer.title] = lLayer.title;
									showDisplayLayerAddFailureWidget(lLayer.title);
								}
							});

							dojo.connect(lLayer, "onLoad", function (error) {
								//selfLocalLayer.publishData({
								//    message : "AllLoaded"
								//});
								if (!(lLayer.title in window.successLayerDictionary)) {
									window.successLayerDictionary[lLayer.title] = lLayer.title;
								}
							});

							if (layer.name) {
								lLayer._titleForLegend = layer.name;
								lLayer.title = layer.name;
								window.hashTitleToEAID[layer.name] = layer.eaID;
								window.hashEAIDToTitle[layer.eaID] = layer.name;
								lLayer.noservicename = true;
							}

							lLayer.on('load', function (evt) {
								evt.layer.name = lOptions.id;
								currentEAID = evt.layer.id.replace(window.layerIdPrefix, "");
								evt.layer.setMinScale(Math.ceil(window.scaleLevelDic[window.hashIDtoCacheLevelNat[currentEAID]]));
								//console.log("on load lOptions.minScale: " +Math.ceil(window.scaleLevelDic[window.hashIDtoCacheLevelNat[currentEAID]]));
								if (evt.layer.id == window.layerIdPrefix + communityBoundaryLayerID) {
									setTimeout(function () {
										var popupsTemplate = {};
										var locationTemplate = new InfoTemplate();
										locationTemplate.setTitle("EnviroAtlas Community Location");
										locationTemplate.setContent(getTextContent);
										var boundaryTemplate = new InfoTemplate();
										boundaryTemplate.setTitle("EnviroAtlas Community Boundary");
										boundaryTemplate.setContent(getTextContent);
										popupsTemplate[0] = {
											infoTemplate: locationTemplate
										};

										popupsTemplate[1] = null;
										evt.layer._titleForLegend = window.communityLayerTitle;
										evt.layer.title = window.communityLayerTitle;
										evt.layer.noservicename = true;
										evt.layer.setInfoTemplates(popupsTemplate);
									}, 1000)
								}
								else if (window.nationalLayerNumber.includes(currentEAID)) {
									setTimeout(function () {
										jimuUtils.adjustMapExtent(selfSimpleSearchFilter.map);
									}, 50)
								}
							});

							lLayer.id = window.layerIdPrefix + layer.eaID.toString();

							this._viewerMap.addLayer(lLayer);
							if (layer.hasOwnProperty('eaScale')) {
								lLayer.eaScale = layer.eaScale;
								if (layer.eaScale == "COMMUNITY") {
									//lLayer.setVisibility(false);
									lLayer.setVisibility(true);
									//turn off the layer when first added to map and let user to turn on
									window.communityLayerNumber.push(layer.eaID.toString());
									setTimeout(function () {
										_addSelectedLayers(layersTobeAdded, communityBoundaryLayerID);
									}, 2000)
								} else {//National
									//lLayer.setVisibility(false);
									lLayer.setVisibility(true);
									window.nationalLayerNumber.push(layer.eaID.toString());
								}
							}
						}//end of if(bNeedToBeAdded)
					}
				}
			}
		};

#### Demographics
    - inputs are renderform
    - Use base json to build a render object 
    - classbreak function applies symbology...applied to map as layerDrawingOptions
    - selected fields build infoTemplates and renderField
    - render object passed to  _genRenderer
    - _genRender makes a wobj from this, which calls _getRenderer, and then makes a dmlayer = new ArcGISDynamicMapServiceLayer 

In [ ]:
robj: {
    bg_max: 16043,
    bg_min: 0,
    blk_max: 2565,
    blk_min: 0,
    classes: "3",
    cnty_max: 3241204,
    cnty_min: 39,
    description: "Number of Households",
    fid: "HSHOLDS",
    fielddesc: "Number of Households",
    fromcolor: "#eff3ff",
    layervisible: true,
    linecolor: b.Color {r: 192, g: 192, b: 192, a: 1},
    linewidth: 1,
    method: "equal-interval",
    mid: "census2010",
    opcvalue: 0.23180175529168823,
    rendertype: "polygon",
    st_max: 12577498.01,
    st_min: 226878.99,
    svcdesc: "2010 Census",
    tocolor: "#08519c",
    tr_max: 16043,
    tr_min: 0,
    }

In [ ]:
_mapRender: function (renderobj) {
                var fielddesc = renderobj.fielddesc;
                var svcdesc = renderobj.svcdesc;
                var opcvalue = renderobj.opcvalue;
                var mapid = renderobj.mid;
                var fieldid = renderobj.fid;
                var layervisible = renderobj.layervisible;
                //console.log(mapid + "; " + fieldid + "; " + renderobj.rendertype)
                var activelayer = this.getActiveLayer(mapid, fieldid);
                var headerfields = _config.demogJSON[mapid].baselayers[activelayer].headerfields;
                renderobj.actlayer = activelayer;
                var alyrindex = null;
                var dataUrl = _config.demogJSON[mapid].layerurl + _config.demogJSON[mapid].service + "/MapServer";
                var renderuniquekey = fieldid + "_" + renderobj.method + "_" + renderobj.classes;
                var orgrender = null;
                var linewidth = renderobj.linewidth;
                var linecolor = renderobj.linecolor;

                var optionsArray = [];
                //var layeridstr = mapid + fieldid + "_map";
                var layeridstr = window.layerIdDemographPrefix + mapid + fieldid + "_map";
                //get renderer for all layers
                var blayers = _config.demogJSON[mapid].baselayers;
                for (var currentLayer in blayers) {
                    orgrender = _config.demogJSON[mapid].baselayers[currentLayer].renderobj[renderuniquekey];
                    alyrindex = _config.demogJSON[mapid].baselayers[currentLayer].layeridx;
                    if (renderobj.rendertype == "polygon") {
                        var mycolors = this.generateColors(renderobj.classes, renderobj.fromcolor, renderobj.tocolor);
                        for (var m = 0; m < orgrender.infos.length; m++) {
                            //console.log(mycolors[m]);
                            orgrender.infos[m].symbol.setColor(mycolors[m]);
                            orgrender.infos[m].symbol.outline.setColor(new Color(linecolor));
                            orgrender.infos[m].symbol.outline.setWidth(linewidth);

                        }
                        renderobj.renderer = orgrender;
                        var drawingOptions = new esri.layers.LayerDrawingOptions();
                        drawingOptions.renderer = orgrender;
                        optionsArray[alyrindex] = drawingOptions;
                    } else {
                        var pntrenderer = new ClassBreaksRenderer();
                        pntrenderer.attributeField = orgrender.attributeField;
                        var fillSymbol = new SimpleFillSymbol();
                        fillSymbol.setColor(new Color([0, 0, 0, 0]));
                        fillSymbol.outline.setColor(new Color([133, 133, 133, .5]));
                        fillSymbol.outline.setWidth(0);

                        pntrenderer.backgroundFillSymbol = fillSymbol;
                        pntrenderer.valueExpressionTitle = fielddesc;
                        var pntcolor = renderobj.circlecolor;

                        var minsize = Number(renderobj.circlemins);
                        var maxsize = Number(renderobj.circlemaxs);
                        var sinterval = (maxsize - minsize) / (Number(renderobj.classes) - 1);
                        for (var m = 0; m < orgrender.infos.length; m++) {
                            var markerSymbol = new SimpleMarkerSymbol();
                            markerSymbol.setColor(new Color(pntcolor));

                            markerSymbol.outline.setColor(new Color(linecolor));
                            markerSymbol.outline.setWidth(linewidth);
                            var s = parseInt(minsize + m * sinterval);
                            //console.log("size: " + s);
                            markerSymbol.setSize(s);
                            pntrenderer.addBreak({
                                minValue: orgrender.infos[m].minValue,
                                maxValue: orgrender.infos[m].maxValue,
                                label: orgrender.infos[m].label,
                                symbol: markerSymbol
                            });

                        }
                        renderobj.renderer = pntrenderer;
                        var drawingOptions = new esri.layers.LayerDrawingOptions();
                        drawingOptions.renderer = pntrenderer;
                        optionsArray[alyrindex] = drawingOptions;
                    }
                }
                alyrindex = _config.demogJSON[mapid].baselayers[activelayer].layeridx;
                window.demographicLayerVisibleIndex = alyrindex;
                //end of getting renderer for all layers


                if (this.map.getLayer(layeridstr)) {
                    var dmlayer = this.map.getLayer(layeridstr);
                    this.map.removeLayer(dmlayer);
                } else {
                    this._serviceWidgets.push(renderobj);
                }
                var dmlayer = new ArcGISDynamicMapServiceLayer(dataUrl, {
                    "id": layeridstr,
                    "opacity": opcvalue,
                    "visible": layervisible
                });

                dmlayer.on('error', function (evt) {
                    alert("We cannot connect to the service, error message: " + evt.error.message);
                });
                this.map.addLayer(dmlayer);
                dmlayer.on("update-start", lang.hitch(this, this.showloading, layeridstr));
                dmlayer.on("update-end", lang.hitch(this, this.hideloading, layeridstr));
                var infostr = "";
                for (var f in headerfields) {
                    var ilabel = headerfields[f];
                    infostr += ilabel + ": ${" + f + "}<br />";

                }
                infostr += fielddesc + ": ${" + fieldid + ": NumberFormat(places:0)}";
                var infoTemplate = new InfoTemplate(fielddesc, infostr);
                var iTemplates = {};
                iTemplates[alyrindex] = { infoTemplate: infoTemplate };

                dmlayer.setInfoTemplates(iTemplates)
                dmlayer.setVisibleLayers([alyrindex]);
                dmlayer.setLayerDrawingOptions(optionsArray);

                //var dtitle = fielddesc + " (" + svcdesc + ")";
                var dtitle = window.demographicsTitlePrefix + fielddesc + " (" + svcdesc + ")";
                //dmlayer.name = fielddesc;
                dmlayer.title = dtitle;
                this.demographicLayerName = dtitle;

                dmlayer.isDynamic = true;
                dmlayer.renderField = fieldid;
                dmlayer.layerType = mapid + "_" + alyrindex;
                dmlayer.renderIndex = alyrindex;

                if (renderobj.whereclause) {
                    var layerdef = [];
                    layerdef[alyrindex] = renderobj.whereclause;
                    dmlayer.setLayerDefinitions(layerdef);
                }
                this.removespining();
            },
 }

In [ ]:
# on click _mapDemog. Calls classbreak           
    _mapDemog: function (e) {
                this.addspining(e);
                var frm = this.renderform;

                var mapid = this.dtype;
                var fieldid = this.demogListNode.value;
                var dmethod = this.classTypeNode.value;
                var dclass = this.classNumNode.value;
                var robj = _config.demogJSON[this.dtype].dynamiclayers[fieldid];
                robj.rendertype = this.rendertype;
                robj.method = dmethod;
                robj.classes = dclass;
                if (this.rendertype == "polygon") {
                    var fcolor = frm.startcolor.value;
                    var ecolor = frm.endcolor.value;
                    robj.fromcolor = fcolor;
                    robj.tocolor = ecolor;
                } else {
                    robj.circlecolor = this.colorpnt.getColor();
                    robj.circlemins = this.minsizeNode.value;
                    robj.circlemaxs = this.maxsizeNode.value;
                }
                robj.mid = mapid;
                robj.fid = fieldid;

                robj.linecolor = this.color1.getColor();
                robj.linewidth = parseInt(this.bWidthNode.value);


                robj.opcvalue = 1 - this.demogsliderNode.value;

                var svcobj = this.serviceNode;
                var svcdesc = svcobj.options[svcobj.selectedIndex].text;
                robj.svcdesc = svcdesc;
                var fieldobj = this.demogListNode;
                var fielddesc = fieldobj.options[fieldobj.selectedIndex].text;
                robj.fielddesc = fielddesc;
                robj.layervisible = true;
                this.classbreak(robj, true);
                var demogSourceForURL = "";
                for (var key in _config.demogJSON) {
                    if (_config.demogJSON.hasOwnProperty(key)) {
                        if (selfDemographic.serviceNode.value == key) {
                            demogSourceForURL = _config.demogJSON[key].service.replace("ejscreen/", "");
                        }
                    }
                }

                var urlParameter = "?demogSource=" + demogSourceForURL + "&demogCategory=" + selfDemographic.demogTypeNode.value + "&demogVariable=" + selfDemographic.demogListNode.value;//?demogSource = census2018acs & demogCategory=Housing& demogVariable=RENT_AMT_3000_3499
                document.getElementById("urlToBeCopied").value = window.location.host + window.location.pathname + urlParameter


            },

In [ ]:
#calls _mapRender
classbreak: function (renderobj) {
                var mapid = renderobj.mid;
                var fieldid = renderobj.fid;
                var activelayer = this.getActiveLayer(mapid, fieldid);
                var descstr = _config.demogJSON[mapid].baselayers[activelayer].level;
                this.levelDiv.innerHTML = descstr;
                renderobj.actlayer = activelayer;
                var renderuniquekey = fieldid + "_" + renderobj.method + "_" + renderobj.classes;
                if (_config.demogJSON[mapid].baselayers[activelayer].renderobj[renderuniquekey]) {
                    this._mapRender(renderobj);
                } else {
                    this._genRender(renderobj);
                    //this._mapRender(renderobj);
                }

            },

# Imports

In [ ]:
import pandas as pd
import json
from itertools import groupby

In [ ]:
pd.set_option("display.max_rows", None) 

In [ ]:
# Input JSON data (from production)
config_path = 'EnviroAtlas/EnviroAtlas_v4/client/src/shared/dataCatalog_config.json'

# Add to map

In [ ]:
with open(config_path, 'r') as f:
    data = json.load(f)

In [ ]:
# Data to drop
keys = [
    'mode', 
    'eaMetric', 
    'eaMetadata', 
    'eaDescription', 
    'eaDfsLink', 
    'sourceType',
    'eaBCSDD',
    'autorefresh',
    'DownloadSource', 
    'eaTags',
    'areaGeog',
    'eaScale',
    'hucNavStats',
    'hucNavStatsUnits',
    'categoryTab',
    'eaTopic',
    'SubLayerIds',
    'SubLayerNames',
    'eaRCA',
    'eaFFM',
    'eaCA',
    'eaCS',
    'eaCPW',
    'eaNHM',
    'eaBC',
    'eaPBS',
    'visible',
    'opacity',
    'IsSubLayer',
    'numDecimal',
    'agoID']

In [ ]:
# Drop all the keys from JSON dataset
for i in data:
    for key in keys:
        i.pop(key, None)

In [ ]:
# Filter data to eaID less than 1000 (i.e. layers) 
data_filtered_layers = [d for d in data if d["eaID"] < 1000]
print(data_filtered_layers)

In [ ]:
output_config_path = 'EnviroAtlas/EnviroAtlas_v4/client/src/shared/dataCatalog_layerConfig.json'

In [ ]:
# Save Layer JSON only
with open(output_config_path, 'w') as file:
    json.dump(data_filtered_layers, file, sort_keys=True)

In [ ]:
filtered_layers_df = pd.DataFrame(data_filtered_layers)

In [ ]:
#filtered_layers_df.IsSubLayer = filtered_layers_df.IsSubLayer.fillna(False)
notTiled = filtered_layers_df.loc[filtered_layers_df['tileLink'] == 'no']
notTiled = notTiled.loc[notTiled['type'] == 'FEATURE']
notTiled = notTiled.drop(columns=['tileLink', 'tileURL', 'cacheLevelNat'])
#notSubLayers
notTiled

In [ ]:
filtered_layers_df

# Data Prep for UI

In [ ]:
with open(config_path, 'r') as f:
    data = json.load(f)

In [ ]:
# Data to drop for UI
keys = [
    'popup', 
    'mode', 
    'numDecimal', 
    'visible', 
    'eaMetric', 
    'eaMetadata', 
    'opacity', 
    'url', 
    'eaDescription', 
    'tileLink', 
    'eaDfsLink', 
    'sourceType',
    'eaBCSDD',
    'autorefresh',
    'type', 
    'DownloadSource', 
    'eaLyrNum',
    'eaTags',
    'areaGeog',
    'eaScale',
    'tileURL',
    'hucNavStats',
    'hucNavStatsUnits',
    'layers',
    'cacheLevelNat',
    'agoID']

In [ ]:
# Drop all the keys from JSON dataset
for i in data:
    for key in keys:
        i.pop(key, None)

# Test process for subtopics with many layers

In [ ]:
# Filter data to eaID greater than or equal to 1000 (i.e. subtopics with layer references) 
data_filtered_subtopics = [d for d in data if d["eaID"] >= 1000]
print(data_filtered_subtopics)

In [ ]:
# Rename columns
rename_columns = {
    "categoryTab" : "category",
    "eaTopic" : "title",
    "SubLayerIds": "A",
    "SubLayerNames": "C"
}
filtered_subtopics_df = pd.DataFrame(data_filtered_subtopics)
filtered_subtopics_df = filtered_subtopics_df.rename(columns=rename_columns)
# filtered_subtopics_df = filtered_subtopics_df.set_index('eaID')
cols=['title', 'category', 'eaBC', 'eaCA', 'eaCPW', 'eaCS', 'eaFFM', 'eaNHM', 'eaRCA', 'name', 'A', 'C', ]
filtered_subtopics_df = filtered_subtopics_df[cols]
filtered_subtopics_df

In [ ]:
# Lump subtopics by eatopic
filtered_subtopics_df_carbon = filtered_subtopics_df.loc[filtered_subtopics_df['title'] == "Soils"]
filtered_subtopics_df_carbon

In [ ]:
explode = filtered_subtopics_df.explode(list('AC'))
layers = explode.iloc[:, 2:].reset_index(drop=True)
# layers = layers.set_index(['name'])
cols = {
    "A": "eaID",
    "C": "title"
}
layers = layers.rename(columns=cols)
layers = layers.astype({'eaID': 'int32'})
layers

In [ ]:
subtopics = layers.to_json(orient='table')

In [ ]:
subtopics

In [ ]:
obj = json.loads(subtopics)
data = obj["data"]
data

In [ ]:
result = []
key_function = lambda x: (x['name'], x['eaBC'], x['eaCA'], x['eaCPW'], x['eaCS'], x['eaFFM'], x['eaNHM'], x['eaRCA'])
data.sort(key = key_function)
for group, checkings in groupby(data, key_function):
        user = {
                'name': group[0],
                'eaBC': group[1],
                'eaCA': group[2],
                'eaCPW': group[3],
                'eaCS': group[4],
                'eaFFM': group[5],
                'eaNHM': group[6],
                'eaRCA': group[7],
                'layers': []
        }
        for title in checkings:
                user['layers'].append(({"title": title['title'], "eaID": title['eaID']}))
        result.append(user)

print(json.dumps(result, indent=2))

# Formalize Process

In [ ]:
def process_old_json(subtopic):
    filtered_subtopics_df_carbon = filtered_subtopics_df.loc[filtered_subtopics_df['title'] == subtopic]
    explode = filtered_subtopics_df_carbon.explode(list('AC'))
    layers = explode.iloc[:, 2:].reset_index(drop=True)
    layers = layers.set_index(['name'])
    cols = {
        "A": "eaID",
        "C": "title"
    }
    layers = layers.rename(columns=cols)
    layers = layers.astype({'eaID': 'int32'})
    subtopics = layers.to_json(orient='table')
    obj = json.loads(subtopics)
    data = obj["data"]
    result = []
    key_function = lambda x: (x['name'], x['eaBC'], x['eaCA'], x['eaCPW'], x['eaCS'], x['eaFFM'], x['eaNHM'], x['eaRCA'])
    data.sort(key = key_function)
    for group, checkings in groupby(data, key_function):
            user = {
                    'name': group[0],
                    'eaBC': group[1],
                    'eaCA': group[2],
                    'eaCPW': group[3],
                    'eaCS': group[4],
                    'eaFFM': group[5],
                    'eaNHM': group[6],
                    'eaRCA': group[7],
                    'layers': []
            }
            for title in checkings:
                    user['layers'].append(({"title": title['title'], "eaID": title['eaID']}))
            result.append(user)

    print(json.dumps(result, indent=2))

### Gather data...brute force copy and paste as "subtopic"...

In [ ]:
process_old_json('Carbon Storage')

In [ ]:
process_old_json('Crop Productivity')

In [ ]:
process_old_json('Ecosystem Markets')

In [ ]:
process_old_json('Energy Potential')

In [ ]:
process_old_json('Engagement with Outdoors')

In [ ]:
process_old_json('Health and Economic Outcomes')

In [ ]:
process_old_json('Land Cover: Near-Water')

In [ ]:
process_old_json('Land Cover: Type')

In [ ]:
process_old_json('Landscape Pattern')

In [ ]:
process_old_json('Livestock and Poultry Production')

In [ ]:
process_old_json('Near-Road Environments')

In [ ]:
process_old_json('Pollutant Reduction: Air')

In [ ]:
process_old_json('Pollutant Reduction: Water')

In [ ]:
process_old_json('Protected Lands')

In [ ]:
process_old_json('Soils')

In [ ]:
process_old_json('Species: At-Risk and Priority')

In [ ]:
process_old_json('Species: Other')

In [ ]:
process_old_json('Water Supply, Runoff, and Flow')

In [ ]:
process_old_json('Water Use')

In [ ]:
process_old_json('Weather and Climate')

In [ ]:
process_old_json('Wetlands and Lowlands')

In [ ]:
process_old_json('Harmful Algal Blooms')

In [ ]:
process_old_json('Impaired Waters')

In [ ]:
process_old_json('National Air Toxics Assessment')

In [ ]:
process_old_json('Pollutants: Nutrients')

In [ ]:
process_old_json('Pollutants: Other')

In [ ]:
process_old_json('Sites Reporting to EPA ')

In [ ]:
process_old_json('Commuting and Walkability')

In [ ]:
process_old_json('Employment')

In [ ]:
process_old_json('Housing and Schools')

In [ ]:
process_old_json('Population Distribution')

In [ ]:
process_old_json('Quality of Life')

In [ ]:
process_old_json('Vacancy')

In [ ]:
process_old_json('Ecological Boundaries')

In [ ]:
process_old_json('Hydrologic Features')

In [ ]:
process_old_json('Political Boundaries')

# TO DO: formalize this bit

In [ ]:
new_df = explode.iloc[:, :2]
new_df = new_df.drop_duplicates()
new_df

In [ ]:
# Create DataCatologItem object
# Define a custom function to create a nested structure
def custom_nested_structure(row):
    return {'Person': {'Name': row['Name'], 'Age': row['Age']}}

In [ ]:
output_config_path = '/src/shared/dataCatalog_testInitSubtopicsOnly.json'

In [ ]:
# Save Subtopic JSON only
with open(output_config_path, 'w') as file:
    json.dump(data_filtered_subtopics, file)

# Test process of subtopic with 1 layer

In [ ]:
# Filter data to eaID less than 1000 (i.e. layers) 
data_filtered_layers = [d for d in data if d["eaID"] < 1000]
print(data_filtered_layers)

In [ ]:
# notSubLayers_dict = filtered_layers_df.loc[filtered_layers_df['IsSubLayer'] == False].to_dict('records')
# notSubLayers_dict

In [ ]:
# Filter out where "IsSubLayer" is true (these are retained in the subtopic data)
filtered_layers_df = pd.DataFrame(data_filtered_layers)
filtered_layers_df.IsSubLayer = filtered_layers_df.IsSubLayer.fillna(False)
notSubLayers = filtered_layers_df.loc[filtered_layers_df['IsSubLayer'] == False]
notSubLayers = notSubLayers.drop(columns=['SubLayerIds', 'IsSubLayer'])
notSubLayers

In [ ]:
# Rename columns
rename_columns = {
    "categoryTab" : "category",
    "eaTopic" : "title",
}
notSubLayers_df = pd.DataFrame(notSubLayers)
notSubLayers_df = notSubLayers.rename(columns=rename_columns)
cols=['title', 'category', 'eaBC', 'eaCA', 'eaCPW', 'eaCS', 'eaFFM', 'eaNHM', 'eaRCA', 'name', 'eaID']
notSubLayers_df = notSubLayers_df[cols]
notSubLayers_df

In [ ]:
# Lump subtopics by eatopic
notSubLayers_subset_df = notSubLayers_df.loc[notSubLayers_df['title'] == "Protected Lands"]
notSubLayers_subset_df

# Merge all subtopic and layers as flat data

In [ ]:
layers = notSubLayers_df.iloc[:, 2:].reset_index(drop=True)
#layers = layers.set_index(['name'])
cols = {
    "A": "eaID",
    "C": "title"
}
layers = layers.rename(columns=cols)
layers2 = layers.astype({'eaID': 'int32'})
layers2['title'] = ""
layers2

In [ ]:
layers2.info()
layers.info()
merge_data = layers.merge(layers2, how='outer')
merge_data

In [ ]:
merge_data.to_csv('EnviroAtlas_v4/client/src/shared/layers.csv')

# Formalize process for subtopics with 1 layer (only difference with process old json is explode step)

In [ ]:
def process_old_json_noSubLayer(subtopic):
    notSubLayers_subtopic_df = notSubLayers_df.loc[notSubLayers_df['title'] == subtopic]
    layers = notSubLayers_subtopic_df.iloc[:, 2:].reset_index(drop=True)
    layers = layers.set_index(['name'])
    cols = {
        "A": "eaID",
        "C": "title"
    }
    layers = layers.rename(columns=cols)
    layers = layers.astype({'eaID': 'int32'})
    subtopics = layers.to_json(orient='table')
    obj = json.loads(subtopics)
    data = obj["data"]
    result = []
    key_function = lambda x: (x['name'], x['eaBC'], x['eaCA'], x['eaCPW'], x['eaCS'], x['eaFFM'], x['eaNHM'], x['eaRCA'])
    data.sort(key = key_function)
    for group, checkings in groupby(data, key_function):
            user = {
                    'name': group[0],
                    'eaBC': group[1],
                    'eaCA': group[2],
                    'eaCPW': group[3],
                    'eaCS': group[4],
                    'eaFFM': group[5],
                    'eaNHM': group[6],
                    'eaRCA': group[7],
                    'layers': []
            }
            for title in checkings:
                    user['layers'].append(({"title": title['name'], "eaID": title['eaID']}))
            result.append(user)

    print(json.dumps(result, indent=2))

In [ ]:
process_old_json_notSubLayers("Protected Lands")

In [ ]:
output_config_path = 'EnviroAtlas_v4/client/src/shared/dataCatalog_testInitLayersOnly.json'

In [ ]:
# Save Layer JSON only
with open(output_config_path, 'w') as file:
    json.dump(notSubLayers_dict, file)